<center><i>Made possible by the Astropy Project and ScienceBetter Consulting through financial support from the Community Software Initiative at the Space Telescope Science Institute.</i></center>

<a id="title_ID"></a>

<a href="http://photutils.readthedocs.io/en/stable/index.html"><img src="https://photutils.readthedocs.io/en/stable/_static/photutils_banner.svg" width=300></a>


# Source Detection with `photutils`
---

##### What is source detection?
In order to do photometry on astronomical image data, one must first determine the locations of the sources in the image. Source detection methods find sources algorithmically, by looking for regions of an image where the signal from a source is statistically higher than the signal from background noise. Some algorithms search for sources whose profiles match specific data models, such as a 2-D Gaussian, while others simply look for local maxima.

The `photutils` package provides a variety of tools that use different detection algorithms to locate sources in an image.

##### What does this tutorial include?
This tutorial covers different tools for source detection with `photutils`, including the following methods:
* Source Detection with `DAOStarFinder`
* Source Detection with `IRAFStarFinder`
* Source Detection with `find_peaks`
* Image Segmentation

##### Which data are used in this tutorial?
We will be manipulating Hubble eXtreme Deep Field (XDF) data, which was collected using the Advanced Camera for Surveys (ACS) on Hubble between 2002 and 2012. The image we use here is the result of 1.8 million seconds (500 hours!) of exposure time, and includes some of the faintest and most distant galaxies that have ever been observed. 

*The methods demonstrated here are available in narrative form within the `photutils.detection` [documentation](http://photutils.readthedocs.io/en/stable/detection.html) and `photutils.segmentation` [documentation](http://photutils.readthedocs.io/en/stable/segmentation.html).*

<div class="alert alert-block alert-warning">
    
<b>Important:</b> Before proceeding, please be sure to update your versions of <code>astropy</code>, <code>matplotlib</code>, and <code>photutils</code>, or this notebook may not work properly. Or, if you don't want to handle packages individually, you can always use (and keep updated!) the <a href="https://astroconda.readthedocs.io">AstroConda</a> distribution.
 
</div>

---

## Import necessary packages

First, let's import packages that we will use to perform arithmetic functions and visualize data:

In [ ]:
from astropy.io import fits
import astropy.units as u
from astropy.nddata import CCDData
from astropy.stats import sigma_clipped_stats, SigmaClip
from astropy.visualization import ImageNormalize, LogStretch
import matplotlib.pyplot as plt
from matplotlib.ticker import LogLocator
import numpy as np
from photutils.background import Background2D, MeanBackground

# Show plots in the notebook
%matplotlib inline

Let's also define some `matplotlib` parameters, such as title font size and the dpi, to make sure our plots look nice. To make it quick, we'll do this by loading a [style file shared with the other photutils tutorials](../photutils_notebook_style.mplstyle) into `pyplot`. We will use this style file for all the notebook tutorials. (See [here](https://matplotlib.org/users/customizing.html) to learn more about customizing `matplotlib`.)

In [ ]:
plt.style.use('../photutils_notebook_style.mplstyle')

## Retrieve data

As described in the introduction, we will be using Hubble eXtreme Deep Field (XDF) data. Since this file is too large to store on GitHub, we will just use `astropy` to directly download the file from the STScI archive: https://archive.stsci.edu/prepds/xdf/ 

(Generally, the best package for web queries of astronomical data is [Astroquery](https://astroquery.readthedocs.io/en/latest/); however, the dataset we are using is a High Level Science Product (HLSP) and thus is not located within a catalog that could be queried with Astroquery.)

Throughout this notebook, we are going to store our images in Python using a `CCDData` object (see [Astropy documentation](http://docs.astropy.org/en/stable/nddata/index.html#ccddata-class-for-images)), which contains a `numpy` array in addition to metadata such as uncertainty, masks, or units. In this case, our data is in electrons per second.

In [ ]:
url = 'https://archive.stsci.edu/pub/hlsp/xdf/hlsp_xdf_hst_acswfc-60mas_hudf_f435w_v1_sci.fits'
xdf_image = CCDData.read(url, unit=u.electron / u.s)

As explained in a [previous notebook](../01_background_estimation/01_background_estimation.ipynb) on background estimation, it is important to **mask** these data, as a large portion of the values are equal to zero. We will mask out the non-data portions of the image array, so all of those pixels that have a value of zero don't interfere with our statistics and analyses of the data. 

In [ ]:
# Define the mask
mask = xdf_image.data == 0
xdf_image.mask = mask

Let's look at the data:

In [ ]:
# Set up the figure with subplots
fig, ax1 = plt.subplots(1, 1, figsize=(8, 8))

# Set up the normalization and colormap
norm_image = ImageNormalize(vmin=1e-4, vmax=5e-2, stretch=LogStretch(), clip=False)
cmap = plt.get_cmap('viridis')
cmap.set_over(cmap.colors[-1])
cmap.set_under(cmap.colors[0])
cmap.set_bad('white') # Show masked data as white
xdf_image_clipped = np.clip(xdf_image, 1e-4, None) # clip to plot with logarithmic stretch

# Plot the data
fitsplot = ax1.imshow(np.ma.masked_where(xdf_image.mask, xdf_image_clipped), 
                      norm=norm_image, cmap=cmap)

# Define the colorbar and fix the labels
cbar = plt.colorbar(fitsplot, fraction=0.046, pad=0.04, ticks=LogLocator(subs=range(10)))
labels = ['$10^{-4}$'] + [''] * 8 + ['$10^{-3}$'] + [''] * 8 + ['$10^{-2}$']
cbar.ax.set_yticklabels(labels)

# Define labels
cbar.set_label(r'Flux Count Rate ({})'.format(xdf_image.unit.to_string('latex')), 
               rotation=270, labelpad=30)
ax1.set_xlabel('X (pixels)')
ax1.set_ylabel('Y (pixels)')

*Tip: Double-click on any inline plot to zoom in.*

---
## Source Detection with `DAOStarFinder`

With the `DAOStarFinder` [class](http://photutils.readthedocs.io/en/stable/api/photutils.DAOStarFinder.html), `photutils` provides users with an easy application of the popular [DAOFIND](http://stsdas.stsci.edu/cgi-bin/gethelp.cgi?daofind) algorithm ([Stetson 1987, PASP 99, 191](http://adsabs.harvard.edu/abs/1987PASP...99..191S)), originally developed at the Dominion Astrophysical Observatory. 

This algorithm detects sources by:
* Searching for local maxima
* Selecting only sources with peak amplitude above a defined threshold
* Selecting sources with sizes and shapes that match a 2-D Gaussian kernel (circular or elliptical)

It returns:
* Location of the source centroid
* Parameters reflecting the source's sharpness and roundness

Generally, the threshold that source detection algorithms use is defined as a multiple of the standard deviation. So first, we need to calculate statistics for the data:

In [ ]:
mean, median, std = sigma_clipped_stats(xdf_image.data, sigma=3.0, maxiters=5, mask=xdf_image.mask)

Now, let's run the `DAOStarFinder` algorithm on our data and see what it finds. 

In [ ]:
from photutils import DAOStarFinder

In [ ]:
daofind = DAOStarFinder(fwhm=5.0, threshold=20. * std)
sources_dao = daofind(np.ma.masked_where(xdf_image.mask, xdf_image))    
print(sources_dao)

In [ ]:
# Set up the figure with subplots
fig, ax1 = plt.subplots(1, 1, figsize=(8, 8))

# Plot the data
fitsplot = ax1.imshow(np.ma.masked_where(xdf_image.mask, xdf_image_clipped), 
                      norm=norm_image, cmap=cmap)
ax1.scatter(sources_dao['xcentroid'], sources_dao['ycentroid'], s=30, marker='o', 
            lw=1, alpha=0.7, facecolor='None', edgecolor='r')

# Define the colorbar and fix the labels
cbar = plt.colorbar(fitsplot, fraction=0.046, pad=0.04, ticks=LogLocator(subs=range(10)))
labels = ['$10^{-4}$'] + [''] * 8 + ['$10^{-3}$'] + [''] * 8 + ['$10^{-2}$']
cbar.ax.set_yticklabels(labels)

# Define labels
cbar.set_label(r'Flux Count Rate ({})'.format(xdf_image.unit.to_string('latex')), 
               rotation=270, labelpad=30)
ax1.set_xlabel('X (pixels)')
ax1.set_ylabel('Y (pixels)')
ax1.set_title('DAOFind Sources')

Let's randomly pull out some of these sources to get a closer look at them.

In [ ]:
fig, axs = plt.subplots(3,3, figsize=(3, 3))
plt.subplots_adjust(wspace=0.1, hspace=0.1)

cutout_size = 20

srcs = np.random.permutation(sources_dao)[:axs.size]
for ax, src in zip(axs.ravel(), srcs):
    slc = (slice(int(src['ycentroid'] - cutout_size), int(src['ycentroid'] + cutout_size)),
           slice(int(src['xcentroid'] - cutout_size), int(src['xcentroid'] + cutout_size)))
    ax.imshow(xdf_image_clipped[slc], norm=norm_image)
    ax.text(2, 2, str(src['id']), color='w', va='top')
    ax.set_xticks([])
    ax.set_yticks([])

<div class="alert alert-block alert-info">

<h3>Exercises:</h3><br>

Re-run the <code>DAOStarFinder</code> algorithm with a smaller threshold (like 5&sigma;), and plot the sources that it finds. Do the same, but with a larger threshold (like 100&sigma;). How did changing the threshold affect the results?

</div>

## Source Detection with `IRAFStarFinder`

Similarly to `DAOStarFinder`, `IRAFStarFinder` is a class that implements a pre-existing algorithm that is widely used within the astronomical community. This class uses the `starfind` [algorithm](http://stsdas.stsci.edu/cgi-bin/gethelp.cgi?starfind)  that was originally part of IRAF.

`IRAFStarFinder` is fundamentally similar to `DAOStarFinder` in that it detects sources by finding local maxima above a certain threshold that match a Gaussian kernel. However, `IRAFStarFinder` differs in the following ways:
* Does not allow users to specify an elliptical Gaussian kernel
* Uses image moments to calculate the centroids, roundness, and sharpness of objects

Let's run the `IRAFStarFinder` algorithm on our data, with the same FWHM and threshold, and see how its results differ from `DAOStarFinder`:

In [ ]:
from photutils import IRAFStarFinder

In [ ]:
iraffind = IRAFStarFinder(fwhm=5.0, threshold=20. * std)
sources_iraf = iraffind(np.ma.masked_where(xdf_image.mask, xdf_image))    
print(sources_iraf)

In [ ]:
# Set up the figure with subplots
fig, ax1 = plt.subplots(1, 1, figsize=(8, 8))

# Plot the data
fitsplot = ax1.imshow(np.ma.masked_where(xdf_image.mask, xdf_image_clipped), 
                      norm=norm_image, cmap=cmap)
ax1.scatter(sources_iraf['xcentroid'], sources_iraf['ycentroid'], s=30, marker='o', 
            lw=1, alpha=0.7, facecolor='None', edgecolor='r')

# Define the colorbar and fix the labels
cbar = plt.colorbar(fitsplot, fraction=0.046, pad=0.04, ticks=LogLocator(subs=range(10)))
labels = ['$10^{-4}$'] + [''] * 8 + ['$10^{-3}$'] + [''] * 8 + ['$10^{-2}$']
cbar.ax.set_yticklabels(labels)

# Define labels
cbar.set_label(r'Flux Count Rate ({})'.format(xdf_image.unit.to_string('latex')), 
               rotation=270, labelpad=30)
ax1.set_xlabel('X (pixels)')
ax1.set_ylabel('Y (pixels)')
ax1.set_title('IRAFFind Sources')

Again, let's randomly select some sources for a closer look:

In [ ]:
fig, axs = plt.subplots(3,3, figsize=(3, 3))
plt.subplots_adjust(wspace=0.1, hspace=0.1)

cutout_size = 20

srcs = np.random.permutation(sources_iraf)[:axs.size]
for ax, src in zip(axs.ravel(), srcs):
    slc = (slice(int(src['ycentroid'] - cutout_size), int(src['ycentroid'] + cutout_size)),
           slice(int(src['xcentroid'] - cutout_size), int(src['xcentroid'] + cutout_size)))
    ax.imshow(xdf_image_clipped[slc], norm=norm_image)
    ax.text(2, 2, str(src['id']), color='w', va='top')
    ax.set_xticks([])
    ax.set_yticks([])

<div class="alert alert-block alert-info">

<h3>Exercises:</h3><br>

Re-run the <code>IRAFStarFinder</code> algorithm with a smaller full-width-half-max (FWHM) – try 3 pixels – and plot the sources that it finds. Do the same, but with a larger FWHM (like 10 pixels). How did changing the FWHM affect the results? What astronomical objects might be better captures by smaller FWHM? Larger?

</div>

## Note: Comparing `DAOStarFinder` and `IRAFStarFinder`

You might have noticed that the `IRAFStarFinder` algorithm only found 211 sources in our data &ndash; 14% of what `DAOStarFinder` found. Why is this?

The answer comes down to the default settings for the two algorithms: (1) there are differences in the upper and lower bounds on the requirements for source roundness and sharpness, and (2) `IRAFStarFinder` includes a minimum separation between sources that `DAOStarFinder` does not have:

|  &nbsp;        | `IRAFStarFinder`   | `DAOStarFinder`   |
|----------------|-------|------|
|   sharplo      |   0.5 |  0.2  |
|   sharphi      |  2.0  | 1.0   |
|   roundlo      |  0.0  |  -1.0  |
|   roundhi      | 0.2   |   1.0 |
|   minsep_fwhm  | 1.5 * FWHM   |   N/A |

Thinking about this, *it then makes sense* that `IRAFStarFinder` would find fewer sources. It has tighter restrictions on source roundness and ``sharplo``, meaning that it eliminates  more elliptical galactic sources (this is the eXtreme Deep Field, after all!), and the minimum separation requirement further rules out sources that are too close to one another.

If we set all these parameters to be equivalent, though, we should find much better agreement between the two methods:

In [ ]:
iraffind_match = IRAFStarFinder(fwhm=5.0, threshold=20. * std, 
                                sharplo=0.2, sharphi=1.0, 
                                roundlo=-1.0, roundhi=1.0,
                                minsep_fwhm=0.0)
sources_iraf_match = iraffind_match(np.ma.masked_where(xdf_image.mask, xdf_image))    
print(sources_iraf_match)

The number of detected sources are in much better agreement now &ndash; 1415 versus 1470 &ndash; but the improved agreement can also be seen by plotting the location of these sources:

In [ ]:
# Set up the figure with subplots
fig, [ax1, ax2] = plt.subplots(1, 2, figsize=(12, 6))
plt.tight_layout()

# Plot the DAOStarFinder data
fitsplot = ax1.imshow(np.ma.masked_where(xdf_image.mask, xdf_image_clipped), 
                      norm=norm_image, cmap=cmap)
ax1.scatter(sources_dao['xcentroid'], sources_dao['ycentroid'], s=30, marker='o', 
            lw=1, alpha=0.7, facecolor='None', edgecolor='r')
ax1.set_xlabel('X (pixels)')
ax1.set_ylabel('Y (pixels)')
ax1.set_title('DAOStarFinder Sources')

# Plot the IRAFStarFinder data
fitsplot = ax2.imshow(np.ma.masked_where(xdf_image.mask, xdf_image_clipped), 
                      norm=norm_image, cmap=cmap)
ax2.scatter(sources_iraf_match['xcentroid'], sources_iraf_match['ycentroid'], 
            s=30, marker='o', lw=1, alpha=0.7, facecolor='None', edgecolor='r')
ax2.set_xlabel('X (pixels)')
ax2.set_title('IRAFStarFinder Sources')

# Define the colorbar
cbar_ax = fig.add_axes([1, 0.09, 0.03, 0.87])
cbar = plt.colorbar(fitsplot, cbar_ax, ticks=LogLocator(subs=range(10)))
labels = ['$10^{-4}$'] + [''] * 8 + ['$10^{-3}$'] + [''] * 8 + ['$10^{-2}$']
cbar.ax.set_yticklabels(labels)
cbar.set_label(r'Flux Count Rate ({})'.format(xdf_image.unit.to_string('latex')), 
               rotation=270, labelpad=30)

Take this example as reminder to be mindful when selecting a source detection algorithm, and when defining algorithm parameters! Don't be afraid to play around with the parameters and investigate how that affects your results.

## Source Detection with `find_peaks`

For more general source detection cases that do not require comparison with models, `photutils` offers the `find_peaks` function. 

This function simply finds sources by identifying local maxima above a given threshold and separated by a given distance, rather than trying to fit data to a given model. Unlike the previous detection algorithms, `find_peaks` does not necessarily calculate objects' centroids. Unless the `centroid_func` argument is passed a function like `photutils.centroids.centroid_2dg` that can handle source position centroiding, `find_peaks` will return just the integer value of the peak pixel for each source.

This algorithm is particularly useful for identifying non-stellar sources or heavily distorted sources in image data.

Let's see how it does:

In [ ]:
from photutils import find_peaks
from photutils.centroids import centroid_2dg

In [ ]:
sources_findpeaks = find_peaks(xdf_image.data, mask=xdf_image.mask, 
                               threshold=20. * std, box_size=30, 
                               centroid_func=centroid_2dg)    
print(sources_findpeaks)

In [ ]:
# Set up the figure with subplots
fig, ax1 = plt.subplots(1, 1, figsize=(8, 8))

# Plot the data
fitsplot = ax1.imshow(np.ma.masked_where(xdf_image.mask, xdf_image_clipped), 
                      norm=norm_image, cmap=cmap)
ax1.scatter(sources_findpeaks['x_peak'], sources_findpeaks['y_peak'], s=30, marker='o', 
            lw=1, alpha=0.7, facecolor='None', edgecolor='r')

# Define the colorbar
cbar = plt.colorbar(fitsplot, fraction=0.046, pad=0.04, ticks=LogLocator(subs=range(10)))
labels = ['$10^{-4}$'] + [''] * 8 + ['$10^{-3}$'] + [''] * 8 + ['$10^{-2}$']
cbar.ax.set_yticklabels(labels)

# Define labels
cbar.set_label(r'Flux Count Rate ({})'.format(xdf_image.unit.to_string('latex')), 
               rotation=270, labelpad=30)
ax1.set_xlabel('X (pixels)')
ax1.set_ylabel('Y (pixels)')
ax1.set_title('find_peaks Sources')

And a closer look:

In [ ]:
fig, axs = plt.subplots(3,3, figsize=(3, 3))
plt.subplots_adjust(wspace=0.1, hspace=0.1)

cutout_size = 20

srcs = np.random.permutation(sources_findpeaks)[:axs.size]
for ax, src in zip(axs.ravel(), srcs):
    slc = (slice(int(src['y_peak'] - cutout_size), int(src['y_peak'] + cutout_size)),
           slice(int(src['x_peak'] - cutout_size), int(src['x_peak'] + cutout_size)))
    ax.imshow(xdf_image_clipped[slc], norm=norm_image)
    src_id = np.where((sources_findpeaks['x_peak'] == src['x_peak']) & 
                      (sources_findpeaks['y_peak'] == src['y_peak']))[0][0]
    ax.text(2, 2, str(src_id), color='w', va='top')
    ax.set_xticks([])
    ax.set_yticks([])

## Comparing Detection Methods

Let's compare how each of these different strategies did.

First, how many sources did each method find?

In [ ]:
print('''DAOStarFinder: {} sources
IRAFStarFinder: {} sources
find_peaks: {} sources'''.format(len(sources_dao), len(sources_iraf_match), len(sources_findpeaks)))

Next, how many of these sources match? We can answer this question by using [sets](https://docs.python.org/3/library/stdtypes.html#set-types-set-frozenset) to compare the centroids of the different sources (rounding to the nearest integer).

In [ ]:
# Make lists of centroid coordinates
centroids_dao = [(x, y) for x, y in sources_dao['xcentroid', 'ycentroid']]
centroids_iraf = [(x, y) for x, y in sources_iraf_match['xcentroid', 'ycentroid']]
centroids_findpeaks = [(x, y) for x, y in sources_findpeaks['x_centroid', 'y_centroid']]

# Round those coordinates to the ones place and convert them to be sets
rounded_centroids_dao = set([(round(x, 0), round(y, 0)) for x, y in centroids_dao])
rounded_centroids_iraf = set([(round(x, 0), round(y, 0)) for x, y in centroids_iraf])
rounded_centroids_findpeaks = set([(round(x, 0), round(y, 0)) for x, y in centroids_findpeaks])

# Examine the intersections of different sets to determine which sources are shared
all_match = rounded_centroids_dao.intersection(rounded_centroids_iraf).intersection(rounded_centroids_findpeaks)
dao_iraf_match = rounded_centroids_dao.intersection(rounded_centroids_iraf)
dao_findpeaks_match = rounded_centroids_dao.intersection(rounded_centroids_findpeaks)
iraf_findpeaks_match = rounded_centroids_iraf.intersection(rounded_centroids_findpeaks)

print('''Matching sources found by:
    All methods: {}
    DAOStarFinder & IRAFStarFinder: {}
    DAOStarFinder & find_peaks: {}
    IRAFStarFinder & find_peaks: {}'''
      .format(len(all_match), len(dao_iraf_match), 
              len(dao_findpeaks_match), len(iraf_findpeaks_match)))

And just for fun, let's plot these matching sources. (The colors chosen to represent different sets are from [Paul Tol's guide for accessible color schemes](https://personal.sron.nl/~pault/).)

In [ ]:
# Set up the figure with subplots
fig, ax1 = plt.subplots(1, 1, figsize=(8, 8))

# Plot the data
fitsplot = ax1.imshow(np.ma.masked_where(xdf_image.mask, xdf_image_clipped), norm=norm_image)
ax1.scatter([x for x, y in list(dao_findpeaks_match)], [y for x, y in list(dao_findpeaks_match)],
            s=30, marker='s', lw=1,  facecolor='None', edgecolor='#EE7733',
            label='Found by DAO and find_peaks')
ax1.scatter([x for x, y in list(dao_iraf_match)], [y for x, y in list(dao_iraf_match)],
            s=30, marker='D', lw=1, facecolor='None', edgecolor='#EE3377',
            label='Found by DAO and IRAF')
ax1.scatter([x for x, y in list(iraf_findpeaks_match)], [y for x, y in list(iraf_findpeaks_match)],
            s=30, marker='o', lw=1, facecolor='None', edgecolor='#0077BB',
            label='Found by IRAF and find_peaks')
ax1.scatter([x for x, y in list(all_match)], [y for x, y in list(all_match)],
            s=30, marker='o', lw=1.2, linestyle=':',facecolor='None', edgecolor='#BBBBBB',
            label='Found by all methods')

# Add legend
ax1.legend(ncol=2, loc='lower center', bbox_to_anchor=(0.5, -0.25))

# Define labels
ax1.set_xlabel('X (pixels)')
ax1.set_ylabel('Y (pixels)')
ax1.set_title('Sources Found by Different Methods')

*Remember that you can double-click on the plot to zoom in and look around!*

## Custom Detection Algorithms

If none of the algorithms we've reviewed above do exactly what you need, `photutils` also provides infrastructure for you to generate and use your own source detection algorithm: the `StarFinderBase` object can be inherited and used to develop new star-finding classes. Take a look at the [documentation](https://photutils.readthedocs.io/en/latest/api/photutils.detection.StarFinderBase.html#photutils.detection.StarFinderBase) for more information.

If you do go that route, remember that `photutils` is open-developed; you would be very welcome to [open a pull request](https://github.com/astropy/photutils/blob/master/CONTRIBUTING.rst) and incorporate your new star finder into the `photutils` source code &ndash; for everyone to use!

---

## Image Segmentation

Beyond traditional source detection methods, an additional option for identifying sources in image data is a process called **image segmentation**. This method identifies and labels contiguous (connected) objects within an image. 

You might have noticed that, in the previous source detection algorithms, large and extended sources are often incorrectly identified as more than one source. Segmentation would label all the pixels within a large galaxy as belonging to the same object, and would allow the user to then measure the photometry, centroid, and morphology of the entire object at once.

#### Creating a `SegmentationImage`

In `photutils`, image segmentation maps are created using the threshold method in the `detect_sources()` function. This method identifies all of the objects in the data that have signals above a determined **`threshold`** (usually defined as a multiple of the standard deviation) and that have more than a defined number of adjoining pixels, **`npixels`**. The data can also optionally be smoothed using a kernel, **`filter_kernel`**, before applying the threshold cut.

The `detect_sources()` function returns a `SegmentationImage` object: an array in which each object is labeled with an integer. As a simple example, a segmentation map containing two distinct sources might look like this:

```
0 0 0 0 0 0 0 0 0 0
0 1 1 0 0 0 0 0 0 0
1 1 1 1 1 0 0 0 2 0
1 1 1 1 0 0 0 2 2 2
1 1 1 0 0 0 2 2 2 2
1 1 1 1 0 0 0 2 2 0
1 1 0 0 0 0 2 2 0 0
0 1 0 0 0 0 2 0 0 0
0 0 0 0 0 0 0 0 0 0
```
where all of the pixels labeled `1` belong to the first source, all those labeled `2` belong to the second, and all null pixels are designated to be background.

Let's see what the segmentation map for our XDF data will look like.

In [ ]:
from photutils import detect_sources
from photutils.utils import make_random_cmap

In [ ]:
# Define threshold and minimum object size
threshold = 5. * std
npixels = 15

# Create a segmentation image
segm = detect_sources(xdf_image.data, threshold, npixels)

print('Found {} sources'.format(segm.max_label))

In [ ]:
# Set up the figure with subplots
fig, [ax1, ax2] = plt.subplots(1, 2, figsize=(12, 6))
plt.tight_layout()

# Plot the original data
fitsplot = ax1.imshow(np.ma.masked_where(xdf_image.mask, xdf_image_clipped), 
                      norm=norm_image, cmap=cmap)
ax1.set_ylabel('Y (pixels)')
ax1.set_title('Original Data')

# Plot the segmentation image
rand_cmap = make_random_cmap(random_state=12345)
rand_cmap.set_under('black')
segplot = ax2.imshow(np.ma.masked_where(xdf_image.mask, segm), vmin=1, cmap=rand_cmap)
ax2.set_xlabel('X (pixels)')
ax2.set_title('Segmentation Map')

# Define the colorbar
cbar_ax = fig.add_axes([1, 0.09, 0.03, 0.87])
cbar = plt.colorbar(segplot, cbar_ax)
cbar.set_label('Object Label', rotation=270, labelpad=30)

Compare the sources in original data to those in the segmentation image. Each color in the segmentation map denotes a separate source.

You can easily see that larger galaxies are shown in the segmentation map as contiguous objects of the same color - for example, the two yellow and pink galaxies near (1200, 2500). Each pixel containing light from the same galaxy has been labeled as belonging to the same object.

For a closer look, let's see what the sources we found with `find_peaks` look like in this segmentation map:

In [ ]:
fig, axs = plt.subplots(3,3, figsize=(3, 3))
plt.subplots_adjust(wspace=0.1, hspace=0.1)

cutout_size = 20

srcs = np.random.permutation(sources_findpeaks)[:axs.size]
for ax, src in zip(axs.ravel(), srcs):
    slc = (slice(int(src['y_peak'] - cutout_size), int(src['y_peak'] + cutout_size)),
           slice(int(src['x_peak'] - cutout_size), int(src['x_peak'] + cutout_size)))
    ax.imshow(segm.data[slc], cmap=rand_cmap, vmin=1, vmax=len(sources_findpeaks))
    src_id = np.where((sources_findpeaks['x_peak'] == src['x_peak']) & 
                      (sources_findpeaks['y_peak'] == src['y_peak']))[0][0]
    ax.text(2, 2, str(src_id), color='w', va='top')
    ax.set_xticks([])
    ax.set_yticks([])

<div class="alert alert-block alert-info">

<h3>Exercises:</h3><br>

Recompute the <code>SegmentationImage</code>, but alter the threshold and the minimum number of pixels in a source. How does changing the threshold affect the results? What about changing the number of pixels?

</div>

#### Analyzing `source_properties`

Once we have a `SegmentationImage` object, `photutils` provides many powerful tools to manipulate and analyze the identified objects. 

Individual objects within the segmentation map can be altered using methods such as `relabel` to change the labels of objects, `remove_labels` to remove objects, or `deblend_sources` to separating overlapping sources that were incorrectly labeled as one source.

However, perhaps the most powerful aspect of the `SegmentationImage` is the ability to create a catalog using `source_properties` to measure the centroids, photometry, and morphology of the detected objects:

In [ ]:
from photutils import source_properties

In [ ]:
catalog = source_properties(xdf_image.data, segm)
table = catalog.to_table()
print(table)
print(table.colnames)

#### Creating apertures from segmentation data

We can use this information to create isophotal ellipses for each identified source. These ellipses can also later be used as photometric apertures!

In [ ]:
from photutils import EllipticalAperture

In [ ]:
# Define the approximate isophotal extent
r = 4.  # pixels

# Create the apertures
apertures = []
for obj in catalog:
    position = (obj.xcentroid.value, obj.ycentroid.value)
    a = obj.semimajor_axis_sigma.value * r
    b = obj.semiminor_axis_sigma.value * r
    theta = obj.orientation.value
    apertures.append(EllipticalAperture(position, a, b, theta=theta))

In [ ]:
# Set up the figure with subplots
fig, ax1 = plt.subplots(1, 1, figsize=(8, 8))

# Plot the data
fitsplot = ax1.imshow(np.ma.masked_where(xdf_image.mask, xdf_image_clipped), 
                      norm=norm_image, cmap=cmap)

# Plot the apertures
for aperture in apertures:
    aperture.plot(color='red', lw=1, alpha=0.7, axes=ax1)

# Define the colorbar
cbar = plt.colorbar(fitsplot, fraction=0.046, pad=0.04, ticks=LogLocator(subs=range(10)))
labels = ['$10^{-4}$'] + [''] * 8 + ['$10^{-3}$'] + [''] * 8 + ['$10^{-2}$']
cbar.ax.set_yticklabels(labels)

# Define labels
cbar.set_label(r'Flux Count Rate ({})'.format(xdf_image.unit.to_string('latex')), 
               rotation=270, labelpad=30)
ax1.set_xlabel('X (pixels)')
ax1.set_ylabel('Y (pixels)')
ax1.set_title('Segmentation Image Apertures')

<div class="alert alert-block alert-info">

<h3>Exercises:</h3><br>

Play with the isophotal extent of the elliptical apertures (defined above as <code>r</code>). Observe how changing this value affects the apertures that are created.

</div>

It is clear that using `photutils` for image segmentation can allow users to generate highly customized apertures &ndash; great for complex data that contain many different kinds of celestial sources.

---
## Conclusions

The `photutils` package provides users with a variety of methods for detecting sources in their data, from familar algorithms such as `DAOFind` and `starfind`, to more complex and customizable image segmentation algorithms. These methods allow for easy creation of a diverse array of apertures that can be used for photometric analysis.

**To continue with this `photutils` tutorial, go on to the [aperture photometry](../03_aperture_photometry/03_aperture_photometry.ipynb) or [PSF photometry notebook](../04_psf_photometry/04_psf_photometry.ipynb).**

---
## Additional Resources
For more examples and details, please visit the [photutils](http://photutils.readthedocs.io/en/stable/index.html) documentation.

---
## About this Notebook
**Authors:** Lauren Chambers (lchambers@stsci.edu), Erik Tollerud (etollerud@stsci.edu), Tom Wilson (towilson@stsci.edu)
<br>**Updated:** May 2019

[Top of Page](#title_ID)
<img style="float: right;" src="https://raw.githubusercontent.com/spacetelescope/notebooks/master/assets/stsci_pri_combo_mark_horizonal_white_bkgd.png" alt="STScI logo" width="200px"/>